<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Assignment: Notebook for Peer Assignment</font></h1>

# Introduction

Using this Python notebook you will:
1. Understand 3 Chicago datasets  
1. Load the 3 datasets into 3 tables in a Db2 database
1. Execute SQL queries to answer assignment questions 

## Understand the datasets 
To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:
1. <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>
1. <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>
1. <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago Crime Data</a>

### 1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2



### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t




### 3. Chicago Crime Data 

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. 

This dataset is quite large - over 1.5GB in size with over 6.5 million rows. For the purposes of this assignment we will use a much smaller sample of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2


### Download the datasets
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the links below to download and save the datasets (.CSV files):
1. __CENSUS_DATA:__ https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv
1. __CHICAGO_PUBLIC_SCHOOLS__  https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv
1. __CHICAGO_CRIME_DATA:__ https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

__NOTE:__ Ensure you have downloaded the datasets using the links above instead of directly from the Chicago Data Portal. The versions linked here are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to complete this assignment.

### Store the datasets in database tables
To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in Week 3 Lab 3, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

<img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg">

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the first dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new tables as folows:
1. __CENSUS_DATA__
1. __CHICAGO_PUBLIC_SCHOOLS__
1. __CHICAGO_CRIME_DATA__

### Connect to the database 
Let us first load the SQL extension and establish a connection with the database

In [1]:
%load_ext sql

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance in first lab in Week 3. From the __uri__ field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://ibm.box.com/shared/static/hzhkvdyinpupm2wfx49lkr71q9swbpec.jpg">

In [2]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://gzv97359:8-n8qx284qf543c1@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
            
import ibm_db

dsn_driver = "ibm_db_sa"
dsn_database = "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net"
dsn_port = "50000"
dsn_protocol = "TCPIP"
dsn_uid = "gzv97359"
dsn_pwd = "8-n8qx284qf543c1"

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

## Problems
Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table

In [31]:
# Rows in Crime table
query = "SELECT COUNT(*) FROM CHICAGO_CRIME_DATA"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

ibm_db.fetch_row(result)
print(ibm_db.result(result, 0))

533


### Problem 2

##### Retrieve first 10 rows from the CRIME table


In [20]:
query = "SELECT * FROM CHICAGO_CRIME_DATA LIMIT 10"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row)
    row = ibm_db.fetch_assoc(result)


{'ID': 3512276, 'CASE_NUMBER': 'HK587712', 'DATE': '08/28/2004 05:50:56 PM', 'BLOCK': '047XX S KEDZIE AVE', 'IUCR': '890', 'PRIMARY_TYPE': 'THEFT', 'DESCRIPTION': 'FROM BUILDING', 'LOCATION_DESCRIPTION': 'SMALL RETAIL STORE', 'ARREST': 'FALSE', 'DOMESTIC': 'FALSE', 'BEAT': 911, 'DISTRICT': 9, 'WARD': 14, 'COMMUNITY_AREA_NUMBER': 58, 'FBICODE': '6', 'X_COORDINATE': 1155838, 'Y_COORDINATE': 1873050, 'YEAR': 2004, 'UPDATEDON': '02/10/2018 03:50:01 PM', 'LATITUDE': '41.80744050', 'LONGITUDE': '-87.70395585', 'LOCATION': '(41.8074405, -87.703955849)'}
{'ID': 3406613, 'CASE_NUMBER': 'HK456306', 'DATE': '06/26/2004 12:40:00 PM', 'BLOCK': '009XX N CENTRAL PARK AVE', 'IUCR': '820', 'PRIMARY_TYPE': 'THEFT', 'DESCRIPTION': '$500 AND UNDER', 'LOCATION_DESCRIPTION': 'OTHER', 'ARREST': 'FALSE', 'DOMESTIC': 'FALSE', 'BEAT': 1112, 'DISTRICT': 11, 'WARD': 27, 'COMMUNITY_AREA_NUMBER': 23, 'FBICODE': '6', 'X_COORDINATE': 1152206, 'Y_COORDINATE': 1906127, 'YEAR': 2004, 'UPDATEDON': '02/28/2018 03:56:25 PM

### Problem 3

##### How many crimes involve an arrest?

In [30]:
query = "SELECT COUNT(*) FROM CHICAGO_CRIME_DATA WHERE ARREST=TRUE"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

ibm_db.fetch_row(result)
print(ibm_db.result(result, 0))

163


### Problem 4

##### Which unique types of crimes have been recorded at GAS STATION locations?


In [24]:
query = "SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION='GAS STATION'"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['PRIMARY_TYPE'])
    row = ibm_db.fetch_assoc(result)


CRIMINAL TRESPASS
NARCOTICS
ROBBERY
THEFT


Hint: Which column lists types of crimes e.g. THEFT?

### Problem 5

##### In the CENUS_DATA table list all Community Areas whose names start with the letter ‘B’.

In [27]:
query = "SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NAME LIKE 'B%'"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['COMMUNITY_AREA_NAME'])
    row = ibm_db.fetch_assoc(result)

Belmont Cragin
Burnside
Brighton Park
Bridgeport
Beverly


### Problem 6

##### Which schools in Community Areas 10 to 15 are healthy school certified?

In [35]:
query = "SELECT NAME_OF_SCHOOL FROM CHICAGO_PUBLIC_SCHOOLS WHERE COMMUNITY_AREA_NUMBER BETWEEN 10 AND 15 AND HEALTHY_SCHOOL_CERTIFIED='Yes'"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['NAME_OF_SCHOOL'])
    row = ibm_db.fetch_assoc(result)

Rufus M Hitch Elementary School


### Problem 7

##### What is the average school Safety Score? 

In [34]:
query = "SELECT AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS"

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

ibm_db.fetch_row(result)
print(ibm_db.result(result, 0))

49.504873


### Problem 8

##### List the top 5 Community Areas by average College Enrollment [number of students] 

In [50]:
query = """
SELECT DISTINCT(COMMUNITY_AREA_NAME)
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE COMMUNITY_AREA_NUMBER IN (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_PUBLIC_SCHOOLS
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY AVG(COLLEGE_ENROLLMENT) DESC
    LIMIT 5
)
""";

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['COMMUNITY_AREA_NAME'])
    row = ibm_db.fetch_assoc(result)

ARCHER HEIGHTS
BELMONT CRAGIN
BRIGHTON PARK
MONTCLARE
WEST ELSDON


### Problem 9

##### Use a sub-query to determine which Community Area has the least value for school Safety Score? 

In [46]:
query = """
SELECT COMMUNITY_AREA_NAME 
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_PUBLIC_SCHOOLS
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY MIN(SAFETY_SCORE)
    LIMIT 1
)
LIMIT 1
""";

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['COMMUNITY_AREA_NAME'])
    row = ibm_db.fetch_assoc(result)

WASHINGTON PARK


### Problem 10

##### [Without using an explicit JOIN operator] Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [56]:
query = """
SELECT CENSUS_DATA.PER_CAPITA_INCOME 
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER IN (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE SAFETY_SCORE = 1
    GROUP BY COMMUNITY_AREA_NUMBER
)
""";

#Execute the statement
result = ibm_db.exec_immediate(conn, query)

row = ibm_db.fetch_assoc(result)
while row != False:
    print(row['PER_CAPITA_INCOME'])
    row = ibm_db.fetch_assoc(result)

13785


Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
